# The Materials API

### Presented by: John Dagdelen

In this lesson, we cover:
* The Materials Project API (MAPI) and its documentation, the [mapidoc](https://github.com/materialsproject/mapidoc).
* Getting your Materials Project API key.
* Using the MPRester to access the MP database.
* A hands-on example of using the API and pymatgen to screen the MP database for interesting materials.

In [ ]:
# This supresses warnings.
import warnings
warnings.filterwarnings('ignore')

# This is a helper function to shorten lists during the 
# live presentation of this lesson for better readability. 
# You can ignore it. 
def shortlist(long_list, n=5):
    print("First {} of {} items:".format(min(n, 5), len(long_list)))
    for item in long_list[0:n]:
        print(item)

***
## Section 0: Getting an API key

The first step to getting started with the API is to get an API key. We do this on the Materials Project website (https://materialsproject.org/dashboard.)

1. Click the `Generate API key` button
2. copy your shiny new key
3. Paste your key in the line below and run the cell.

In [ ]:
!pmg config --add PMG_MAPI_KEY <your API key>

## Section 1: The MAPIDOC

The [mapidoc](https://github.com/materialsproject/mapidoc) is a key source of information regarding the Materials Project API. It should be the first thing you consult whenever you are having trouble with the API. Let's take a look!

***
## Section 2: Basic Queries In the Web Browser

To request data from the Materials Project, you will need to make requests to our API. To do this, you could simply make a GET request through your web browser, providing your API key as an argument. 

For example, 

`https://www.materialsproject.org/rest/v2/materials/mp-1234/vasp?API_KEY=<your api key>`


For example, 

`https://www.materialsproject.org/rest/v2/materials/mp-1234/vasp?API_KEY=<your api key>`

returns the following JSON document:
```
{"response": [{"energy": -26.94573468, "energy_per_atom": -4.49095578, "volume": 116.92375473740876, "formation_energy_per_atom": -0.4835973866666663, "nsites": 6, "unit_cell_formula": {"Al": 4.0, "Lu": 2.0}, "pretty_formula": "LuAl2", "is_hubbard": false, "elements": ["Al", "Lu"], "nelements": 2, "e_above_hull": 0, "hubbards": {}, "is_compatible": true, "spacegroup": {"source": "spglib", "symbol": "Fd-3m", "number": 227, "point_group": "m-3m", "crystal_system": "cubic", "hall": "F 4d 2 3 -1d"}, "task_ids": ["mp-1234", "mp-925833", "mp-940234", "mp-940654"], "band_gap": 0.0, "density": 6.502482433523648, "icsd_id": null, "icsd_ids": [608375, 57958, 608376, 608372, 608371, 608370], "cif": "# generated using pymatgen\ndata_LuAl2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   5.48873905\n_cell_length_b   5.48873905\n_cell_length_c   5.48873905\n_cell_angle_alpha   60.00000005\n_cell_angle_beta   60.00000003\n_cell_angle_gamma   60.00000007\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   LuAl2\n_chemical_formula_sum   'Lu2 Al4'\n_cell_volume   116.92375474\n_cell_formula_units_Z   2\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Al  Al1  1  0.500000  0.500000  0.500000  1\n  Al  Al2  1  0.500000  0.500000  0.000000  1\n  Al  Al3  1  0.000000  0.500000  0.500000  1\n  Al  Al4  1  0.500000  0.000000  0.500000  1\n  Lu  Lu5  1  0.875000  0.875000  0.875000  1\n  Lu  Lu6  1  0.125000  0.125000  0.125000  1\n", "total_magnetization": 0.0012519, "material_id": "mp-1234", "oxide_type": "None", "tags": ["High pressure experimental phase", "Aluminium lutetium (2/1)"], "elasticity": null, "full_formula": "Lu2Al4"}], "valid_response": true, "created_at": "2018-08-08T18:52:53.042666", "version": {"db": "3.0.0", "pymatgen": "2018.7.23", "rest": "2.0"}, "copyright": "Materials Project, 2018"}
```

For obvious reasons, typing these kinds of urls into your web browser is not an ideal way to request MP data. Instead, we should try to access the API programatically with python. Let's do the same request that we did above using Python's *requests* library. 

### Making Requests With Python

***
## Section 3: The MPRester

In this section we will:

* Open the pymatgen.MPRester web documentation.
* Create our first instance of an MPRester object.
* Get our feet wet with calling a few of the MPRester's "specialty" methods.



#### Background and Documentation

* Code connects to the MP Database through REST requests.
* Pymatgen's MPRester class is helpful for accessing our API in python.
* The [documentation](http://pymatgen.org/pymatgen.ext.matproj.html#pymatgen.ext.matproj.MPRester) for the MPRester is *very* helpful. Let's take a look!

#### Background and Documentation

REST is a widely used type of standardization that allows different computer systems to work together. In RESTful systems, information is organized into resources, each of which is uniquely identified via a uniform resource identifier (URI). Since MAPI is a RESTful system, users can interact with the MP database regardless of their computer system or programming language (as long as it supports basic http requests.)

To facilitate researchers in using our API, we implemented a convenient wrapper for it in the Python Materials Genomics (pymatgen) library called the `MPRester`. You can find the relevant pymatgen documentation for it [here](http://pymatgen.org/pymatgen.ext.matproj.html?highlight=mprester#pymatgen.ext.matproj.MPRester).


### Starting up an instance of the MPRester

We'll import the MPRester and create an instance of it. 

*Note: You may need to use your API key as an input argument if it has not been pre-configured.*

However, we recommend that you use the “with” context manager to ensure that sessions are properly closed after usage:

### MPRester Methods:

The MPRester has many methods that you might want to use in your research. For example, there is a method to get the bandstructure for a material, `get_bandstructure_by_material_id`. 

Let's use this method and the following bandstructure plotting function to get and plot a bandstructure for mp-1234:

In [ ]:
### Don't edit this code ####
from pymatgen.electronic_structure.plotter import BSPlotter
# Helpful function for plotting a bandstructure. 
def plot_bandstructure(bs):
    BSPlotter(bs).get_plot().show() 
#############################

In [ ]:
# Excercise: Use the MPRester's get_bandstructure_by_material_id method to 
# get a bandstructure from the MP Database and plot it using the
# plot_bandstructure functin defined above.


There's also a method to get MPIDs for a formula or chemical system called `get_materials_ids`.

In [ ]:
with MPRester() as mpr:
    # You can pass in a formula to get_materials_ids

    # Or you can pass in a "chemsys" such as "Li-Fe-P-O"


### Excercise: 

Imagine you want to get the **structure** for the multiferroic material BiFeO3 (**`mp-24932`**) and suggest some **substrates** for growing it.

Use methods of the MPRester to get this information from the Materials Project API. 

Hints:
* `MPRester.get_structure_by_material_id`
* `MPRester.get_substrates` 

In [ ]:
# Get the structure for BiFeO3 (mp-24932) and 
# suggest some substrates for growing it.


At this point, you should be comfortable with:

* Finding documentation on the MPRester. 
* Creating an instance of the MPRester.
* Using methods of the MPRester.

***
## Section 4: Using the MPRester.query method.

The MPRester also has a very powerful method called `query`, which allows us to perform sophisticated searches on the database. The `query` method uses MongoDB's [query syntax](https://docs.mongodb.com/manual/tutorial/query-documents/). In this syntax, query submissions have two parts: a set of criteria that you want to base the search on (in the form of a python dict), and a set of properties that you want the database to return (in the form of either a list or dict). 

You will probably find yourself using the MPRester's query method frequently. 

The general structure of a MPRester query is:
                            
                            mpr.query(criteria, properties)




The general structure of a MPRester query is:
                            
                            mpr.query(criteria, properties)

* `criteria` is usually a string or a dict. 
* `properties` is always a list of strings

Let's try out some queries to learn how it works! 

First, we'll query for $SiO_2$ compounds by chemical formula through 'pretty_formula'.

If we investigate the object that the query method returns, we find that it is a list of dicts. Furthermore, we find that the keys of the dictionaries are the very same keywords that we passed to the query method as the `properties` argument.

In fact, if you are just looking for materials based on formula/composition/stoichiometry, there is an easier way to use the `query` method: just pass in a string as the criteria! 

You can even use **wildcard** characters in your searches. For example, if we want to find all $ABO_3$ compounds in the Materials Project:

### Excercise: 

There are 296 variants of $SiO_2$ in the MP database, but how many $Si_xO_y$ compounds are there in the Materials Project?

Hint: 
* Query using a `chemsys` string instead of a formula.

## Let's do an in-depth example: 

Say we want to find the distribution of crystal systems (cubic, tetragonal, monoclinic, etc) within all SiO$_2$ compounds. How can we do that? 

1. Collect all entries in the MP database with the formula SiO2
2. Collect their crystal systems
3. Plot a bar chart

## MongoDB Operators

Above, we specified the chemical formula SiO$_2$ for our query. This is an example of, the "specify" operator. However, MongoDB's syntax also includes other [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors), allowing us to bulid complex conditionals into our queries. These all start with the "$" character.



A recent paper by McEnany et. al. proposes a novel ammonia synthesis process based on the electrochemical cycling of lithium ([link](http://pubs.rsc.org/en/content/articlelanding/2017/ee/c7ee01126a#!divAbstract)). As an exercise, let's use some of MongoDB's operators and ask the database for nitrides of alkali metals.

In [ ]:
# Find all nitrides of alkali metals


In [ ]:
#Bonus short way to do this with wildcards


We can also perform the same query, but ask the database to only return compounds with energies above the hull less than 10 meV/atom by using the "less than" operator, "`$lt`". (The energy above the convex hull gives us a sense of how stable a compound is relative to other compounds with the same composition.) 

***
## Section 5: Using the MPRester and Pymatgen to Find Materials With Exotic Mechanical Properties

The tetragonal SiO$_2$ polymorph $\alpha$-cristobalite is one of the very few crystalline materials known to have a negative average Poisson's ratio, which means that its cross-section expands under tensile strain rather than contracting. This property can be extremely useful in a variety of applications such as scratch-resistant coatings and high-toughness ceramics. 

Why does $\alpha$-cristobalite exhibit this property while other materials do not? The prevailing hypothesis is that $\alpha$-cristobalite's negative Poisson's ratio is a result of its crystal structure. If that's the case, then perhaps we can find other materials with this exotic property by looking for materials with similar structures and then calculating their Poisson's ratios.

First, we should find the ground state structure for $\alpha$-cristobalite, which has the spacegroup $P4_12_12$ [92]. Remember, stable materials have *low* energy above hull (`e_above_hull`).

We found [(mp-6945)](https://materialsproject.org/materials/mp-6945/), the right material!

For our search, we want to start with a set of structures that are:
* Computationally tractable (not too many sites, <50 is a good number)
* Not too unlikely to be synthesizable (small energy above hull, i.e. <100 meV)

Let's construct this query to get the structures for all materials that match these criteria:

The next step is to compare all of these materials to $\alpha$-cristobalite. We'll need something that can tell us whether two structures are similar. Luckily for us, the pymatgen `StructureMatcher` does just that!

We know that the high-temperature phase of cristobalite, $\beta$-cristobalite [(mp-546794)](https://materialsproject.org/materials/mp-546794/), has a very similar structure to $\alpha$-cristobalite. Let's see if the structure matcher agrees.

Unfortunately, the default settings of the structure matcher are too strict for our purposes. We want a comparison engine that will return True if two structures are similar to each other, not just those that are exactly the same. 

To solve this problem, we can instantiate our comparison engine with looser tolerances and use a species-agnostic `FrameworkComparator` from pymatgen, which allows us to compare structures across different chemistries.

Just to make sure we haven't increased the tolerances too much, let's try it against a random compound to make sure it's not matching un-similar structures. 

Imagine that we have an experimental colleague, Soren Tsarpinski, who is an expert at synthesizing vanadate compounds. We have a hunch that some of the vanadates coming out of Dr. Tsarpinski's lab might have similar structures to $\alpha$-cristobalite and therefore might have negative Poisson's ratios. Let's see if we're right:

Ok, let's see if we have any auxetic vanadates!